# check gpu resources

In [48]:
# gpu resources
! nvidia-smi

Fri Mar 26 03:19:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.39       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:19:00.0 Off |                  N/A |
| 63%   83C    P2   201W / 250W |  10890MiB / 11019MiB |     84%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 67%   

# testing ManTraNet on different datasets

In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import requests
import sys

from PIL import Image
from io import BytesIO
from matplotlib import pyplot

In [2]:
# ManTraNet paths
manTraNet_root = './'
manTraNet_srcDir = os.path.join( manTraNet_root, 'src' )
sys.path.insert( 0, manTraNet_srcDir )
manTraNet_modelDir = os.path.join( manTraNet_root, 'pretrained_weights' )

# setup dataset paths

In [3]:
# sample ManTraNet datasets included in repo
manTraNet_dataDir = os.path.join( manTraNet_root, 'data' )
sample_file = os.path.join( manTraNet_dataDir, 'samplePairs.csv' )
print(sample_file)
assert os.path.isfile( sample_file ), "ERROR: can NOT find sample data, check `manTraNet_root`"
with open( sample_file ) as IN :
    sample_pairs = [line.strip().split(',') for line in IN.readlines() ]
L = len(sample_pairs)
print("INFO: in total, load", L, "samples")
    
def get_a_random_pair() :
    idx = np.random.randint(0,L)
    return ( os.path.join( manTraNet_dataDir, this ) for this in sample_pairs[idx] ) 

./data/samplePairs.csv
INFO: in total, load 72 samples


In [45]:
# CG-1050 dataset
mfc_data = os.path.join(manTraNet_root, 'openmfc_data')
cg_1050 = os.path.join(mfc_data, 'CG_1050')
cg_1050_description = os.path.join(cg_1050, 'DESCRIPTION')

cg_1050_mask = os.path.join(cg_1050, 'MASK')
cg_1050_original = os.path.join(cg_1050, 'ORIGINAL')
cg_1050_tampered = os.path.join(cg_1050, 'TAMPERED')

# load dataset description

In [5]:
description_path = os.path.join(cg_1050_description, 'Dataset_description_v2.csv')

df = pd.read_csv(description_path)

In [6]:
df.head()

,Unnamed: 0,PHOTO NAME (Original),IMAGE DESCRIPTION,PHOTO PLACE,FOLDER NAME,TAMPERING TYPE,PHOTO NAME (Tampered),OBJECT,LOCATION,FOLDER NAME.1,PHOTO NAME (Mask),Pixel changed
0,0,Im_1,Color 3456x4608,Classroom,T_1,Copy-move,Im1_cm1.jpg,telecomunication tower,Middle left,Mask1,Mask1_cm1.png,Black
1,1,Im_1,Color 3456x4608,Classroom,T_1,Copy-move,Im1_cm2.jpg,Window,Middle left,Mask1,Mask1_cm2.png,Black
2,2,Im_1,Color 3456x4608,Classroom,T_1,Copy-move,Im1_cm3.jpg,Tank,Middle left,Mask1,Mask1_cm3.png,Black
3,3,Im_1,Color 3456x4608,Classroom,T_1,Copy-move,Im1_cm4.jpg,Floor,Upper left,Mask1,Mask1_cm4.png,Black
4,4,Im_1,Color 3456x4608,Classroom,T_1,Cut-paste,Im1_cmfr1.jpg,Church tower,Middle left,Mask1,Mask1_cmfr1.png,Black


# build dataset list

In [7]:
df["tamper_path"] = df["FOLDER NAME"] + '/' + df["PHOTO NAME (Tampered)"]
df["mask_path"] = df["FOLDER NAME.1"] + '/' + df["PHOTO NAME (Mask)"]

In [22]:
class Sample:
    def __init__(self, name, tampered, masks):
        self.name = name
        self.tampered = tampered
        self.masks = masks

In [30]:
dataset = []
unique_images = list(df['PHOTO NAME (Original)'].unique())

for image in unique_images:
    tampered = list(df.loc[df['PHOTO NAME (Original)'] == image]['tamper_path'])
    masks = list(df.loc[df['PHOTO NAME (Original)'] == image]['mask_path'])
    sample = Sample(name=image,
                   tampered=tampered,
                   masks=masks)
    dataset.append(sample)

# dataset should have 100 original images (there are more tampered and masks)
assert len(dataset) == 100

In [37]:
print(dataset[0].name)
print(dataset[0].tampered)
print(dataset[0].masks)

Im_1
['T_1/Im1_cm1.jpg', 'T_1/Im1_cm2.jpg', 'T_1/Im1_cm3.jpg', 'T_1/Im1_cm4.jpg', 'T_1/Im1_cmfr1.jpg', 'T_1/Im1_col1.jpg', 'T_1/Im1_col2.jpg', 'T_1/Im1_col3.jpg', 'T_1/Im1_r1.jpg', 'T_1/Im1_r2.jpg', 'T_1/Im1_r3.jpg']
['Mask1/Mask1_cm1.png', 'Mask1/Mask1_cm2.png', 'Mask1/Mask1_cm3.png', 'Mask1/Mask1_cm4.png', 'Mask1/Mask1_cmfr1.png', 'Mask1/Mask1_col1.png', 'Mask1/Mask1_col2.png', 'Mask1/Mask1_col3.png', 'Mask1/Mask1_r1.png', 'Mask1/Mask1_r2.png', 'Mask1/Mask1_r3.png']


In [56]:
print(len(dataset[0].tampered))
print(len(dataset[0].masks))

11
11


In [62]:
def get_a_random_sample():
    idx = np.random.randint(0,len(dataset))
    sample = dataset[idx]
    original_path = os.path.join(cg_1050_original, sample.name + '.jpg')
    idx_mt = np.random.randint(0,len(sample.tampered)) # each tampered image has a mask, so lengths are the same
    mask_path = os.path.join(cg_1050_mask, sample.masks[idx_mt])
    tampered_path = os.path.join(cg_1050_tampered, sample.tampered[idx_mt])
    return (original_path, mask_path, tampered_path)

# Load A Pretrained ManTraNet Model

In [31]:
import modelCore
manTraNet = modelCore.load_pretrain_model_by_index( 4, manTraNet_modelDir )

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:522: FutureWarn

INFO: use activation in the last CONV=None
INFO: unfreeze feature extraction part, trainable=True


In [32]:
# ManTraNet Architecture 
print(manTraNet.summary(line_length=120))

________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
img_in (InputLayer)                    (None, None, None, 3)      0                                                     
________________________________________________________________________________________________________________________
Featex (Model)                         (None, None, None, 256)    3675181       img_in[0][0]                            
________________________________________________________________________________________________________________________
outlierTrans (Conv2D)                  (None, None, None, 64)     16384         Featex[1][0]                            
________________________________________________________________________________________________________________________
bnorm (BatchNormalization)      

In [33]:
# Image Manipulation Classification Network
IMCFeatex = manTraNet.get_layer('Featex')
print(IMCFeatex.summary(line_length=120))

________________________________________________________________________________________________________________________
Layer (type)                                          Output Shape                                    Param #           
image_in (InputLayer)                                 (None, None, None, 3)                           0                 
________________________________________________________________________________________________________________________
b1c1 (CombinedConv2D)                                 (None, None, None, 16)                          525               
________________________________________________________________________________________________________________________
b1c2 (Conv2DSymPadding)                               (None, None, None, 32)                          4640              
________________________________________________________________________________________________________________________
b2c1 (Conv2DSymPadding)         

# test samples from cg-1050

In [34]:
from datetime import datetime 
def read_rgb_image( image_file ) :
    rgb = cv2.imread( image_file, 1 )[...,::-1]
    return rgb
    
def decode_an_image_array( rgb, manTraNet ) :
    x = np.expand_dims( rgb.astype('float32')/255.*2-1, axis=0 )
    t0 = datetime.now()
    y = manTraNet.predict(x)[0,...,0]
    t1 = datetime.now()
    return y, t1-t0

def decode_an_image_file( image_file, manTraNet ) :
    rgb = read_rgb_image( image_file )
    mask, ptime = decode_an_image_array( rgb, manTraNet )
    return rgb, mask, ptime.total_seconds()

In [64]:
for k in range(8) :
    # get a sample
    #forged_file, original_file = get_a_random_pair()
    orig_file, mask_file, tampered_file = get_a_random_sample()
    print(orig_file)
    print(mask_file)
    print(tampered_file)
    print(os.path.exists(orig_file))
    print(os.path.exists(mask_file))
    print(os.path.exists(tampered_file))
    break
    
    # load the original image just for reference
    ori = read_rgb_image( original_file )
    # manipulation detection using ManTraNet
    rgb, mask, ptime = decode_an_image_file( forged_file, manTraNet ) 
    # show results
    pyplot.figure( figsize=(15,5) )
    pyplot.subplot(131)
    pyplot.imshow( ori )
    pyplot.title('Original Image')
    pyplot.subplot(132)
    pyplot.imshow( rgb )
    pyplot.title('Forged Image (ManTra-Net Input)')
    pyplot.subplot(133)
    pyplot.imshow( mask, cmap='gray' )
    pyplot.title('Predicted Mask (ManTra-Net Output)')
    pyplot.suptitle('Decoded {} of size {} for {:.2f} seconds'.format( os.path.basename( forged_file ), rgb.shape, ptime ) )
    pyplot.show()

./openmfc_data/CG_1050/ORIGINAL/Im_39.jpg
./openmfc_data/CG_1050/MASK/Mask39/Mask39_col1.png
./openmfc_data/CG_1050/TAMPERED/T_39/Im39_col1.jpg
True
True
True
